##### 타이타닉 생존자 예측

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic') # seaborn에 기본적으로 있는 타이타닉 데이터셋 불러오기
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


###### 1. 데이터 전처리 

- Feature Selection: 필요한 항목만 골라내기

In [6]:
# 필요로 하는 데이터 가져오기
df = df[['survived',  'pclass', 'sex', 'age', 'sibsp', 'parch','fare','embarked','deck']]
print(df.shape)
df.head(3)

(891, 9)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN


- 결측지 처리

In [5]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [7]:
# age = 평균으로 대체
df.age.fillna(df.age.mean(), inplace = True)
df.age.isnull().sum()

0

In [11]:
df.embarked.value_counts() #embarked 열의 최빈값 확인

S    646
C    168
Q     77
Name: embarked, dtype: int64

In [8]:
# embarked = 최빈값으로 대체 (가장 많이 나오는 값)
df.embarked.fillna('S', inplace = True)
df.embarked.isna().sum()

0

In [12]:
# deck: 열 삭제
df.drop(columns = ['deck'], inplace=True)
df.isna().sum().sum()

0

- 카테고리 값(sex, embarked)을 숫자로 변환

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [14]:
df.sex = le.fit_transform(df.sex) # df의 sex 컬럼을 숫자로 변환
df.embarked = le.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


###### 2. Train, Test 데이터 셋으로 나누기

In [15]:
# X와 y를 numpy 배열로 변환
X = df.iloc[:,1:].values # numpy 배열로 변환 (2차원 배열)
y = df.survived.values # 1차원  벡터
X.shape, y.shape

((891, 7), (891,))

In [17]:
# y값의 분포
#df.survived.value_counts()
np.unique(y, return_counts=True)

(array([0, 1]), array([549, 342]))

In [18]:
# train, test dataset 세팅
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 2022)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [19]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([439, 273]))

###### 3. RandomForeset 모델로 학습
 - 여러 개의 결정 트리를 이용한 학습 방법

In [20]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2022)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2022,
 'verbose': 0,
 'warm_start': False}

In [21]:
# 학습
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2022)

###### 4. 모델 예츨 및 평가

In [22]:
rfc.score(X_test, y_test)

0.8324022346368715

###### 5. 3,4번 방법 대신 GridSearchCV 사용

In [23]:
params = {'max_depth': [2,4,6,8], 'min_samples_split':[2,4,6]}

In [25]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, params, scoring = 'accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [26]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 4}

In [27]:
params = {'max_depth': [3,4,5], 'min_samples_split':[3,4,5]}

In [29]:
grid_rf = GridSearchCV(rfc, params, scoring = 'accuracy', cv=5)
%time grid_rf.fit(X_train, y_train)

grid_rf.best_params_

CPU times: user 7.66 s, sys: 59.7 ms, total: 7.72 s
Wall time: 8.13 s


{'max_depth': 4, 'min_samples_split': 3}

In [30]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8212290502793296

###### 6. test 데이터에 적용

In [31]:
X_test[25], y_test[25]

(array([ 3.  ,  1.  , 45.  ,  0.  ,  0.  ,  8.05,  2.  ]), 1)

In [33]:
best_rf.predict(X_test[25].reshape(1,-1))[0] # 실제 예측해보기

0

###### 7. 엉터리 분류기

In [34]:
# 여성의 생존률 계산
df.groupby('sex')['survived'].mean()

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

In [36]:
df.groupby(['sex', 'pclass'])['survived'].mean()

sex  pclass
0    1         0.968085
     2         0.921053
     3         0.500000
1    1         0.368852
     2         0.157407
     3         0.135447
Name: survived, dtype: float64

In [41]:
# 엉터리 분류기
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
  # fit(), predict() method만 재정의 = overriding
  def fit(self, X,y):
    pass
  def predict(self, X):
    pred = np.zeros(X.shape[0]) # X 데이터의 행의 개수만큼 0으로 초기화한 배열을 만든다
    for i in range(X.shape[0]):
      if X[i,1] == 0: # 여성이면
        pred[i] = 1 # 생존한 것으로 간주하겠다
    return pred

In [42]:
my_clf = MyClassifier()
my_clf.fit(X_train, y_train)
pred_my = my_clf.predict(X_test)

In [47]:
X_test[8,1], pred_my[6]

(0.0, 0.0)

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

In [49]:
pred_rf = best_rf.predict(X_test)
sdf =pd.DataFrame({'y_test':y_test, 'RF':pred_rf, 'MY':pred_my})
sdf.head()

,y_test,RF,MY
0,1,0,0.0
1,0,0,0.0
2,1,0,1.0
3,0,0,0.0
4,0,0,0.0


- 모델의 성능을 평가할 때 무조건적으로 정확도를 사용하는 것은 지양해야 한다

#### 오차 행렬

In [50]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_rf)

array([[103,   7],
       [ 25,  44]])

In [51]:
confusion_matrix(y_test, pred_my)

array([[96, 14],
       [24, 45]])

In [52]:
from sklearn.metrics import precision_score, recall_score

In [54]:
# 정밀도(precision_score (TP/FP+TP))
precision_score(y_test, pred_rf), precision_score(y_test, pred_my)

(0.8627450980392157, 0.7627118644067796)

In [55]:
# 재현율(recall_score(TP/FN+TP))
recall_score(y_test, pred_rf),recall_score(y_test, pred_my)

(0.6376811594202898, 0.6521739130434783)

In [56]:
# F1 score (정밀도와 재현율의 조화 평균)
from sklearn.metrics import f1_score
f1_score(y_test, pred_rf), f1_score(y_test, pred_my)

(0.7333333333333333, 0.703125)

In [57]:
# AUC score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred_rf), roc_auc_score(y_test, pred_my)

(0.787022397891963, 0.7624505928853755)